In [5]:
import os
import json
import time
import datetime
import requests
import pandas as pd
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [43]:
start_date = datetime.date(2020, 12, 18)
end_date = datetime.date(2021, 1, 18)
delta = datetime.timedelta(days=1)
articles=[]
while start_date <= end_date:
    print(start_date)
    gnews_api = os.environ["gnews_api"]
    gnews_url =f"https://gnews.io/api/v4/search?q=bitcoin&in=cryptocurrency&from={start_date}T00:01:36Z&to={start_date}T23:59:36Z&lang=en&token={gnews_api}"
    response = requests.get(gnews_url)
    data = response.json()
    articles.append(data)
    start_date += delta
    time.sleep(4)

2020-12-18
2020-12-19
2020-12-20
2020-12-21
2020-12-22
2020-12-23
2020-12-24
2020-12-25
2020-12-26
2020-12-27
2020-12-28
2020-12-29
2020-12-30
2020-12-31
2021-01-01
2021-01-02
2021-01-03
2021-01-04
2021-01-05
2021-01-06
2021-01-07
2021-01-08
2021-01-09
2021-01-10
2021-01-11
2021-01-12
2021-01-13
2021-01-14
2021-01-15
2021-01-16
2021-01-17
2021-01-18


In [46]:
with open('articles_1.txt', 'w') as outfile:
    json.dump(articles, outfile)

In [54]:
df = pd.DataFrame({"Publish Date": [], "title": [], "description": [], "text": []})
for x in articles:
    for article in x['articles']:
        df = df.append({'Publish Date':article["publishedAt"],
                     'title': article["title"], 
                     'description': article["description"], 
                     'text': article["content"]}, 
                     ignore_index=True)

In [70]:
df['Publish Date']=pd.to_datetime(df['Publish Date'], infer_datetime_format=True).dt.date
df["All Text"]= df['title'] + " " + df['description'] + " " + df['text']
bitcoin_news= df.groupby(by="Publish Date").sum()

In [ ]:
import re
import nltk
from nltk.corpus import stopwords, reuters
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('reuters')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()

In [100]:
# Import the libraries for sentiment scoring using Vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [101]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/alhamduliallah/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [102]:
# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

In [103]:
# Define two lists to store vader sentiment scoring
y_vader_pred = []
y_vader_prob = []

In [106]:
# Score sentiment of test set using Vader
for comment in bitcoin_news["All Text"]:
    y_vader_prob.append(analyzer.polarity_scores(comment)["pos"])
    sentiment_score = analyzer.polarity_scores(comment)["compound"]
    if sentiment_score >= 0.1:
        y_vader_pred.append(1)
    else:
        y_vader_pred.append(0)

In [114]:
bitcoin_sentiment = bitcoin_news.drop(["title", "description", "Title & Description", "text"], axis=1)
bitcoin_sentiment["Sentiment Score"]=y_vader_pred
bitcoin_sentiment["Polarity Score"]=y_vader_prob
bitcoin_sentiment.head()

,All Text,Sentiment Score,Polarity Score
Publish Date,,,
2020-12-18,How to invest in bitcoin: The major ways to bu...,1,0.092
2020-12-19,How will Bitcoin shift the power in the coffee...,0,0.049
2020-12-20,"Tesla's Elon Musk asks about converting ""large...",1,0.050
2020-12-21,Ripple to face SEC suit over XRP cryptocurrenc...,1,0.056
2020-12-22,SEC Sues Ripple Over XRP Cryptocurrency The ag...,1,0.075


In [ ]:
# Need to clean text, text was not cleaned prior to getting scores! So clean before rescoring.